In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model.classification import *
from src import *
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
files = [
    '20190819-Kutna Hora-L1-out-MVI_0007',
    '20190819-Kutna Hora-L3-in-MVI_0005',
    '20190819-Kutna Hora-L3-out-MVI_0008',
    '20190819-Kutna Hora-L4-in-MVI_0013',
    '20190819-Kutna Hora-L6-out-MVI_0017',
    '20190819-Kutna Hora-L7-out-MVI_0032',
    '20190819-Kutna Hora-L8-in-MVI_0045',
    '20190819-Kutna Hora-L9-in-MVI_0043',
    '20190819-Kutna Hora-L10-out-SDV_1888',
    '20190819-Kutna Hora-L13-in-MVI_0006',
    '20190819-Kutna Hora-L13-out-MVI_0018',
    '20190819-Kutna Hora-L14-out-MVI_0005',
    '20190819-Kutna Hora-L15-out-MVI_0012',
    '20190819-Kutna Hora-L18-in-MVI_0030',
    '20190819-Ricany-L2-in-MVI_0006',
    '20190819-Ricany-L2-out-MVI_0005',
    '20190819-Ricany-L3-in-MVI_0006',
    '20190819-Ricany-L6-in-MVI_0008',
    '20190819-Ricany-L6-out-MVI_0011',
    '20190819-Ricany-L7-in-MVI_0008',
    '20190819-Ricany-L7-out-MVI_0013',
    '20190819-Ricany-L8-in-MVI_0009',
    '20190819-Ricany-L8-out-MVI_0013',
    '20190819-Ricany-L9-in-MVI_0008',
    '20190819-Ricany-L9-out-MVI_0011'
]

In [4]:
datapool = DataPool(files, window_length=6, split_ratio=0.75, tqdm=tqdm)

Video loading:   0%|          | 0/25 [00:00<?, ?it/s]

In [5]:
uuid = 1637706373
# suffix = 'diff'
# suffix = 'mae'
suffix = 'interval'
model, params = load_model(uuid, suffix)
transform = create_transformation(params)

In [6]:
def test(is_trn=True):
    outputs = []

    for video in tqdm(datapool):
        n_events = video.get_events_count(is_trn)
        from_time, till_time = video.get_from_till_time(is_trn)

        predictions = validate(video.signal, model, transform, params, from_time=from_time, till_time=till_time, classification=True)
        mae = get_diff(video.signal, video.events, predictions, params, from_time, till_time)

        n_predicted = predictions.sum()
        rvce = np.abs(n_predicted - n_events) / n_events
        error = n_predicted - n_events
        outputs.append([f'{rvce:.3f}', error, n_events, f'{mae:.3f}', f'[{from_time:.0f}, {till_time:.0f}]  {video.file}'])

    outputs = np.array(outputs)
    idx = np.argsort(outputs[:, 0].astype(float))
    outputs = outputs[idx]

    rvce = outputs[:, 0].astype(float).mean()

    print('rvce', rvce)
    
    return outputs

def print_outputs(outputs):
    print(' # |  rvce | error | n_events |     mae | file')
    print('– ' * 50)
    for i, row in enumerate(outputs):
        row[0]
        print(f'{i + 1:>2} | {row[0]} | {row[1]:>5} | {row[2]:>8} | {row[3]:>7} | {row[4]}')
    print('– ' * 50)
    mean_rvce = outputs[:, 0].astype(float).mean()
    mean_error = outputs[:, 1].astype(int).mean()
    mean_n_events = outputs[:, 2].astype(int).mean()
    mean_mae = outputs[:, 3].astype(float).mean()
    print(f'   | {mean_rvce:.3f} | {mean_error:>5} | {mean_n_events:>8} | {mean_mae:7.4} | ')
    
    

In [7]:
val_outputs = test(is_trn=False)
trn_outputs = test(is_trn=True)

  0%|          | 0/25 [00:00<?, ?it/s]

rvce 0.05552000000000001


  0%|          | 0/25 [00:00<?, ?it/s]

rvce 0.03548


In [10]:
print_outputs(trn_outputs)

 # |  rvce | error | n_events |     mae | file
– – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – 
 1 | 0.000 |     0 |        6 |   0.000 | [0, 1620]  20190819-Kutna Hora-L13-in-MVI_0006
 2 | 0.000 |     0 |       49 |   0.008 | [0, 1584]  20190819-Ricany-L9-in-MVI_0008
 3 | 0.000 |     0 |      218 |   1.096 | [0, 942]  20190819-Ricany-L3-in-MVI_0006
 4 | 0.004 |     1 |      258 |   1.220 | [0, 1524]  20190819-Ricany-L8-in-MVI_0009
 5 | 0.004 |     2 |      504 |   2.639 | [0, 1362]  20190819-Kutna Hora-L18-in-MVI_0030
 6 | 0.005 |    -1 |      217 |   0.694 | [0, 1590]  20190819-Ricany-L7-out-MVI_0013
 7 | 0.006 |     1 |      173 |   0.746 | [0, 1560]  20190819-Kutna Hora-L3-out-MVI_0008
 8 | 0.014 |    -3 |      208 |   3.103 | [0, 1566]  20190819-Kutna Hora-L1-out-MVI_0007
 9 | 0.016 |     5 |      317 |   2.485 | [0, 1422]  20190819-Ricany-L7-in-MVI_0008
10 | 0.022 |    13 |      578 |   7.375 | [0, 1566]  20190819-Ricany-L6-out-

In [11]:
print_outputs(val_outputs)

 # |  rvce | error | n_events |     mae | file
– – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – – 
 1 | 0.000 |     0 |        2 |   0.000 | [1554, 2052]  20190819-Kutna Hora-L15-out-MVI_0012
 2 | 0.000 |     0 |       42 |   0.899 | [1530, 2064]  20190819-Kutna Hora-L14-out-MVI_0005
 3 | 0.000 |     0 |        1 |   0.000 | [1680, 2046]  20190819-Kutna Hora-L13-out-MVI_0018
 4 | 0.000 |     0 |        2 |   0.000 | [1620, 2040]  20190819-Kutna Hora-L13-in-MVI_0006
 5 | 0.000 |     0 |       11 |   0.171 | [1566, 2058]  20190819-Kutna Hora-L7-out-MVI_0032
 6 | 0.000 |     0 |       12 |   0.286 | [1554, 2058]  20190819-Ricany-L9-out-MVI_0011
 7 | 0.000 |     0 |        4 |   0.000 | [1482, 2046]  20190819-Kutna Hora-L6-out-MVI_0017
 8 | 0.006 |    -1 |      161 |   1.078 | [1596, 2058]  20190819-Ricany-L6-in-MVI_0008
 9 | 0.013 |     1 |       77 |   2.772 | [1590, 2064]  20190819-Ricany-L7-out-MVI_0013
10 | 0.025 |     4 |      157 |   